In [1]:
import re 
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException, WebDriverException
import time

# ChromeDriver 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 구글 맵 페이지 열기 및 '뱅뱅막국수' 검색
driver.get("https://www.google.com/maps")
try:
    # 검색어 입력 및 검색
    search_box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="searchboxinput"]'))
    )
    search_box.send_keys("파스쿠찌 뱅뱅사거리")
    search_box.send_keys(Keys.ENTER)
except Exception as e:
    print(f"검색창을 찾을 수 없습니다: {e}")
    driver.quit()
    exit()

# 페이지 로드 대기
time.sleep(5)

In [2]:
try:
    # 첫 번째 검색 결과를 JavaScript로 클릭
    first_result = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '(//*[@class="hfpxzc"])[1]'))
    )
    driver.execute_script("arguments[0].click();", first_result)
except Exception as e:
    print(f"첫 번째 검색 결과를 클릭할 수 없습니다: {e}")
    



첫 번째 검색 결과를 클릭할 수 없습니다: Message: 
Stacktrace:
	GetHandleVerifier [0x00D2EBD3+24307]
	(No symbol) [0x00CB8D74]
	(No symbol) [0x00B9C323]
	(No symbol) [0x00BDDC86]
	(No symbol) [0x00BDDECB]
	(No symbol) [0x00C1B9F2]
	(No symbol) [0x00BFFED4]
	(No symbol) [0x00C19579]
	(No symbol) [0x00BFFC26]
	(No symbol) [0x00BD219C]
	(No symbol) [0x00BD311D]
	GetHandleVerifier [0x00FD8D93+2818227]
	GetHandleVerifier [0x0103542E+3196750]
	GetHandleVerifier [0x0102D9D2+3165426]
	GetHandleVerifier [0x00DCDA70+675216]
	(No symbol) [0x00CC1B3D]
	(No symbol) [0x00CBEA18]
	(No symbol) [0x00CBEBB5]
	(No symbol) [0x00CB1640]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77DD809E+286]
	RtlGetAppContainerNamedObjectPath [0x77DD806E+238]



In [3]:
# 리뷰 페이지 열기
try:
    review_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(@aria-label, "리뷰 더보기")]'))
    )
    review_button.click()
    time.sleep(5)  # 리뷰 페이지가 완전히 로드되도록 추가 대기
except Exception as e:
    print(f"리뷰 페이지를 열 수 없습니다: {e}")
    driver.quit()
    exit()

In [5]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

# 리뷰 섹션의 컨테이너를 찾음
try:
    review_container = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div:nth-child(2) > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde'))
    ) #목록이 하나가 아닌경우: #QA0Szd > div > div > div.w6VYqd > div.bJzME.Hu9e2e.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde
    
    # 스크롤을 반복적으로 내리기
    max_wait_time = 5  # 최대 대기 시간 시작 (초)
    while True:
        # 현재 로드된 리뷰 수 확인
        reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
        previous_count = len(reviews)
        
        # 스크롤을 아래로 내림
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", review_container)
        
        # 새로운 리뷰가 로드될 때까지 대기
        try:
            WebDriverWait(driver, max_wait_time).until(
                lambda driver: len(driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')) > previous_count
            )
            # 리뷰가 로드되면 대기 시간을 초기화
            max_wait_time = 5
        except TimeoutException:
            # 새로운 리뷰가 로드되지 않으면 더 이상 로드할 리뷰가 없는 것으로 간주하고 종료
            print("더 이상 로드할 리뷰가 없습니다.")
            break

        # 스크롤이 아래로 내려갈수록 로딩 시간이 길어질 수 있으므로, 실패 시 대기 시간 증가
        max_wait_time += 1  # 한 번 실패할 때마다 대기 시간을 1초씩 늘림 (필요 시 조정 가능)

    print("모든 리뷰를 끝까지 로드했습니다.")
except Exception as e:
    print(f"스크롤 중 오류 발생: {e}")


더 이상 로드할 리뷰가 없습니다.
모든 리뷰를 끝까지 로드했습니다.


In [6]:

# 리뷰 데이터 수집
reviews_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf.fontBodyMedium')
for review in reviews:
    try:
        # '자세히' 버튼 클릭
        try:
            more_button = review.find_element(By.XPATH, './/span[2]/button')
            more_button.click()
            time.sleep(1)  # '자세히' 클릭 후 리뷰 로드 대기
        except NoSuchElementException:
            pass  # '자세히' 버튼이 없으면 무시하고 진행
        
        # 리뷰 작성자 ID
        reviewer_id = review.find_element(By.CSS_SELECTOR, '.d4r55').text
        # 리뷰 작성 날짜
        date = review.find_element(By.CSS_SELECTOR, '.rsqaWe').text
        # 리뷰 내용
        review_text = review.find_element(By.CSS_SELECTOR, '.wiI7pd').text
        reviews_list.append({
            "ID": reviewer_id,
            "Date": date,
            "Review": review_text
        })
    except Exception as e:
        print(f"리뷰 수집 중 오류: {e}")
        continue

# 데이터프레임으로 변환 및 출력
reviews_df = pd.DataFrame(reviews_list)
print(reviews_df)

# CSV 파일로 저장
#reviews_df.to_csv('bangbang_makguksu_reviews.csv', index=False, encoding='utf-8-sig')

# 드라이버 종료
#driver.quit()

리뷰 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":".wiI7pd"}
  (Session info: chrome=130.0.6723.117); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00D2EBD3+24307]
	(No symbol) [0x00CB8D74]
	(No symbol) [0x00B9C323]
	(No symbol) [0x00BDDC86]
	(No symbol) [0x00BDDECB]
	(No symbol) [0x00BD3CF1]
	(No symbol) [0x00BFFED4]
	(No symbol) [0x00BD3C14]
	(No symbol) [0x00C00124]
	(No symbol) [0x00C19579]
	(No symbol) [0x00BFFC26]
	(No symbol) [0x00BD219C]
	(No symbol) [0x00BD311D]
	GetHandleVerifier [0x00FD8D93+2818227]
	GetHandleVerifier [0x0103542E+3196750]
	GetHandleVerifier [0x0102D9D2+3165426]
	GetHandleVerifier [0x00DCDA70+675216]
	(No symbol) [0x00CC1B3D]
	(No symbol) [0x00CBEA18]
	(No symbol) [0x00CBEBB5]
	(No symbol) [0x00CB1640]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77D

In [7]:
reviews_df

,ID,Date,Review
0,이용욱,4년 전,"2층 흡연공간, 강남역 대로변보다 조용해서 미팅, 공부 그리고 휴식하기 좋습니다!!"
1,최현주,2년 전,"좁은 감이 있지만, 그래서 대화집중이 잘 되는 곳! 흡연실도 따로 있음!"
2,JAY LIM,9달 전,이층 흡연실 좋아요^^
3,In Chu,2년 전,직원분이 언제나 친절하게 응대해주셔서 편안한 곳 입니다.
4,박정환,5년 전,강남에서 주차무료인 커피숍이란!!
5,ming ming,3년 전,좋음
6,Hatip Kemal,1년 전,강남 지역의 멋진 카페 가게


In [8]:
reviews_df.to_csv('pascucci_reviews_google.csv', index=False, encoding='utf-8-sig')